In [ ]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import scipy.stats as st

In [ ]:
working_dir = "/Users/filippo/Developer/tesi"
os.chdir(working_dir)
dirs = os.listdir("data")

In [ ]:
normalisation_str='counts'

In [ ]:
common_tissues = ['Bronchus and lung', 'Brain', 'Breast', 'Ovary', 'Kidney', 'Colon', 'Corpus uteri','Adrenal gland','Skin']

In [ ]:
samples = pd.read_csv("files.txt", index_col=[0])
samples.head()

In [ ]:
primaries = samples['primary_site'].unique()
diseases = samples['disease_type'].unique()

In [ ]:
df = pd.read_csv(("%s/mainTable.csv"%working_dir), index_col=[0])
#df = df.to_sparse(fill_value=0.)
df.fillna(value=0., inplace=True)
df.head()

## Tissue U

In [ ]:
tissue = 'Colon'

In [ ]:
sample_list = samples[samples['primary_site']==tissue].index.values
subdf = df.loc[:,sample_list]

In [ ]:
O = []
for g in subdf.index.values:
    o = len([fpkm for fpkm in subdf.loc[g].values if fpkm > 1e-1])
    if o>1:
        O.append(o)

In [ ]:
bins = 20
rang = (0-0.5/20,1+0.5/20)
fig = plt.figure()
plt.hist(np.array(O,dtype=float)/len(sample_list), histtype='step', lw=4, density=True, bins=bins, range=rang)
plt.title(tissue, fontsize=18)
plt.xlabel('$O_i$', fontsize=16)
plt.ylabel('#')
plt.show()
fig.savefig("U_%s.png"%tissue)

## average tissues

In [ ]:
#df_tissue = pd.DataFrame(index=df.index, columns=np.unique(samples['primary_site'].values), dtype=float)
df_tissue = pd.read_csv("results/proteincoding/mainTable_tissues.csv", index_col=[0])
df_tissue.fillna(value=0.,inplace=True)
df_tissue.head()

In [ ]:
#for i,g in enumerate(df_tissue.index.values):
#    gdf = df.loc[g,:]
#    print(g,i)
#    for tissue in df_tissue.columns.values:
#        sample_list = samples[samples['primary_site']==tissue].index.values
#        subdf = gdf.loc[sample_list]
#        df_tissue.at[g,tissue]=np.average(np.array(subdf.values, dtype=float))

In [ ]:
#df_tissue.to_csv("mainTable_tissues.csv")

In [ ]:
gene='ENSG00000198888'

In [ ]:
data = np.array(df.loc[gene,:].values,dtype=float)
x = np.linspace(0,data.max()+10)
mu = np.average(data)
var = np.var(data)
a = mu*mu/var
b = var/mu
fig=plt.figure()
ax=fig.subplots()
ax.set_title('Expression across all tissues: %s'%gene, fontsize=18)
plt.hist(data, histtype='step', lw=2,bins=x, label='gene', density=True)
plt.plot(x, st.gamma.pdf(x/b, a,0,1)/b)
ax.set_xlabel('%s'%normalisation_str,fontsize=16)
ax.set_ylabel('#')
#ax.set_xscale('log')
#ax.set_yscale('log')
ax.set_xlim(5e-2,data.max())
plt.show()
fig.savefig("plot/%s_%s_alltissues.png"%(normalisation_str,gene))

In [ ]:
fig=plt.figure(figsize=(25,20))
N=len(common_tissues)
axs=fig.subplots(3,N/3)
for i,tissue in enumerate(common_tissues):
    ax=axs[i/3][i%(N/3)]
    ax.set_title('%s: %s'%(tissue,gene), fontsize=13)
    sample_list = samples[samples['primary_site']==tissue].index.values
    subdf = df.loc[gene,sample_list]
    data = subdf.fillna(value=0.).values
    #data = data * np.average(data) / np.var(data)
    x = np.linspace(0,data.max()+10)
    mu = np.average(data)
    var = np.var(data)
    a = mu*mu/var
    b = var/mu
    ax.hist(data,density=True,label="%s [%d]"%(tissue, len(data)),histtype='step',lw=1.5)
    ax.plot(x,st.gamma.pdf(x,a,0,b))
    ax.set_xlim(xmax=data.max())
    ax.set_xlabel('%s'%normalisation_str,fontsize=13)
    ax.set_ylabel('#')
    ax.set_xlim(5e-2,data.max())
    ax.legend(fontsize=16)
plt.show()
fig.savefig("plot/%s_%s_per_tissue.png"%(normalisation_str,gene))

In [ ]:
fig=plt.figure()
ax=fig.subplots()
ax.set_title('Expression per tissue: %s'%gene, fontsize=18)
data = np.array(df_tissue.loc[gene,:].values,dtype=float)
x = np.linspace(0,data.max())
mu = np.average(data)
var = np.var(data)
a = mu*mu/var
b = var/mu
plt.hist(data, histtype='step', lw=2, label='gene', density=True, bins=10)
plt.plot(x, st.gamma.pdf(x/b,a)/b)
ax.set_xlabel('%s'%normalisation_str,fontsize=16)
ax.set_ylabel('#')
#ax.set_xscale('log')
#ax.set_yscale('log')
plt.show()
fig.savefig("plot/%s_%s_across_tissue.png"%(normalisation_str,gene))

## integral

In [ ]:
fig=plt.figure(figsize=(15,10))
ax=fig.subplots()
ax.set_title('%s sorted integral'%normalisation_str, fontsize=18)
#for tissue in ['Blood','Heart, mediastinum, and pleura','Brain','Skin','Testis','Thyroid gland']:
for tissue in np.concatenate((common_tissues,['Blood','Heart, mediastinum, and pleura','Brain','Skin','Testis','Thyroid gland'])):
    subdf = df_tissue.sort_values(by=tissue, axis=0, ascending=False).loc[:,tissue]
    norm = np.sum(subdf.values[:])
    if norm!=0:
        ax.plot(np.cumsum(subdf.values[:])/norm,label=tissue, lw=3)
ax.set_xscale('log')
ax.set_xlabel('Number of genes', fontsize=18)
ax.set_ylabel('Fraction of total tissue expression', fontsize=18)
ax.set_xlim((1,2e4))
plt.legend()
plt.show()
fig.savefig("plot/proteincoding/tissue/fraction_of_trascriptome.pdf")

## Mean Variance

In [ ]:
tissue = 'Breast'
subdf = df.loc[:,samples[samples['primary_site']==tissue].index.values]
means = subdf.apply(np.nanmean, axis=1).values
variances = subdf.apply(np.nanvar, axis=1).values
distrs = subdf.sum(axis=0)

In [ ]:
plt.hist(distrs)
print(np.mean(distrs),np.var(distrs))

In [ ]:
x_lin = np.logspace(np.log10(means[means.nonzero()].min()),np.log10(means[means.nonzero()].max()), dtype=float,num=50)
fig=plt.figure(figsize=(15,4))
plt.scatter(means, variances, c='b')
plt.plot(x_lin[-40:],np.power(x_lin[-40:],2), 'g-', lw=3.5, label='$<%s>^2$'%normalisation_str)
plt.plot(x_lin[:20],x_lin[:20], 'r-', lw=3.5, label='$<%s>$'%normalisation_str)

plt.xlabel("$<%s>$"%normalisation_str, fontsize=16)
plt.ylabel("$\sigma^2_{%s}$"%normalisation_str, fontsize=16)
plt.xscale('log')
plt.yscale('log')
plt.title(tissue,fontsize=18)
plt.xlim(means[means.nonzero()].min()/5,np.power(10,np.log10(means.max())+1))
plt.ylim((variances[variances.nonzero()].min()/10,np.power(10,np.log10(variances.max())+1)))
plt.legend(fontsize=16)
plt.show()
fig.savefig("varmean_loglog_%s.png"%tissue)

In [ ]:
cv2 = [variances[i]/(np.power(mean,2)) for i,mean in enumerate(means) if mean>0]
fig=plt.figure(figsize=(15,4))
plt.scatter(means[means.nonzero()], cv2, c='b')
plt.plot(x_lin[:30],1./x_lin[:30], 'g-', lw=3.5, label='Poisson')
plt.plot(x_lin[-30:],[1 for _ in x_lin[-30:]], 'r-', lw=3.5, label='Taylor')
plt.plot(x_lin,[len(subdf.columns)-1 for _ in x_lin], '--', lw=3.5, label='bound')


plt.title(tissue,fontsize=18)
plt.xlabel("$<%s>$"%normalisation_str, fontsize=16)
plt.ylabel("$cv^2$", fontsize=16)
plt.xscale('log')
plt.yscale('log')
plt.xlim(means[means.nonzero()].min()/5,np.power(10,np.log10(means.max())+1))
plt.ylim(variances[variances.nonzero()].min()/10,len(subdf.columns)*10)
plt.legend(fontsize=16)
plt.show()
fig.savefig("cvmean_loglog_%s.png"%tissue)